# Навигация и краткое описание

**Цель:** Сравнительный анализ линейных моделей машинного обучения из библиотеки sklearn для решения задач классификации.([Dataset](https://www.kaggle.com/datasets/mohankrishnathalla/diabetes-health-indicators-dataset))

**Рассматриваемые модели:**
- [LogisticRegression](#logisticregression)
- Linear SVM:
    - [LinearSVC](#linearsvc)
- Универсальный оптимизатор:
    - [SGDClassifier](#sgdclassifier)
- [LinearDiscriminantAnalysis](#lineardiscriminantanalysis)

# Подготовка к работе с моделями

In [1]:
import pandas as pd 
import sklearn

import sys
sys.path.append('..')
from utils import load_processed_data, ModelsClassificationHistory

In [2]:
# Заранее установим несколько констант
RANDOM_STATE = 42

# Загрузка данных
X_train, X_test, y_train, y_test = load_processed_data()
history_models = ModelsClassificationHistory()

# Создание моделей

## LogisticRegression

In [3]:
from sklearn.linear_model import LogisticRegression

Логистическая регрессия - это линейная модель для задач классификации, которая строит линейную зависимость между признаками и вероятностью принадлежности к целевому классу. В отличие от линейной регрессии, она предсказывает не числовое значение, а вероятность от 0 до 1.

Для бинарной классификации логистическая регрессия сначала вычисляет линейную комбинацию признако (логит), а затем преобразует её в вероятность с помощью сигмоидной функции:

$$ 
    z = w_0 + w_1 * x_1 + w_2 * x_2 + \dots + w_n * x_n
$$

$$ 
    P(y=1) = \sigma(z) = \frac{1}{1+e^{-z}}
$$
где:
- $P(y=1)$ - вероятность принадлежности к классу 1
- $w_0$ - смещение (bias)
- $w_1, w_2, \dots, w_n$ - коэффициенты при признаках
- $x_1, x_2, \dots, x_n$ - значения признаков
- $\sigma(z)$ - сигмоидная функция


Графическое представление формулы представлено ниже:

![Геометрический смысл логистической регрессии](../data/images/logRegression/geometric_meaning.png)

In [4]:
LR_standard = LogisticRegression(random_state=RANDOM_STATE)
LR_standard.fit(X_train, y_train)
y_pred = LR_standard.predict(X_test)
y_pred_proba = LR_standard.predict_proba(X_test)  # Вероятности для метрик

c:\Users\sinde\OneDrive\Рабочий стол\classic_ml_skills\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sinde\OneDrive\Рабочий стол\classic_ml_skills\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
history_models.add_model(
    LR_standard, 
    "linear_model", 
    LR_standard.get_params(), 
    "Логистическая регрессия, стандартные параметры", 
    y_true=y_test, 
    y_pred=y_pred,
    y_pred_proba=y_pred_proba
)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,Accuracy,Precision,Recall,F1,ROC-AUC
0,LogisticRegression,linear_model,"{'C': 1.0, 'class_weight': None, 'dual': False...","Логистическая регрессия, стандартные параметры",0.997,0.997,0.997,0.997,None


## SVC

SVC(Support Vector Classification) - идея этой модели заключается в преобразование данных в пространство более высокой размерности с помощью ядерного трюка([kernel trick](https://dzen.ru/a/YPNphUmJnDlJkmrt)), где классы становятся линейно разделимы.

![Пример разделения данных](../data/images/svm/kernel_trick.png)

У SVC есть несколько типов ядер:
* Линейное
    * Плюсы: быстрый и интерпретируемый
    * Минусы: только линейные границы
* Полиномиальное
    * Плюсы: способен аппроксимировать сложные нелинейные зависимости
    * Минусы: сложно интерприетировать и высокая сложность
* Сигмоидальное ядро
    * Плюсы: похожа на двухслойную NN
    * Минусы: чувствительна к масштабированию данных

Разбор параметров `sklearn.svm.SVC`
```python
svc_detailed = SVC(
    # === Выбор ядра ===
    kernel='rbf',           # 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'
    
    # === Параметры ядер ===
    degree=3,               # Степень для poly ядра
    gamma='scale',          # 'scale', 'auto', или числовое значение
    coef0=0.0,              # Параметр для poly/sigmoid ядер
    
    # === Регуляризация ===
    C=1.0,                  # Штраф за ошибки (меньше → более гладкая граница)
    
    # === Производительность ===
    cache_size=200,         # Размер кэша (MB)
    max_iter=-1,            # -1 = неограничено (опасно для больших данных!)
    
    # === Решающая функция ===
    decision_function_shape='ovr',  # 'ovr' или 'ovo' для многоклассовой классификации
    
    random_state=42
)
```

### LinearSVC

Почему мы разбирали SVC, а сейчас мы используем LinearSVC?

LinearSVC - это тот же SVC, но быстрый и гибкий для линейной классификации, тогда как SVC(kernel='linear') - это общее решение, которое медленнее и менее оптимизировано.

In [6]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# Базовая реализация
linear_svc = LinearSVC(
    random_state=42,
    max_iter=1000  # Увеличиваем итерации для сходимости
)

linear_svc.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
y_proba = linear_svc.decision_function(X_test)

c:\Users\sinde\OneDrive\Рабочий стол\classic_ml_skills\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
history_models.add_model(
    linear_svc, 
    "linear_model", 
    linear_svc.get_params(), 
    "линейная SVC", 
    y_true=y_test, 
    y_pred=y_pred,
    y_pred_proba=y_pred_proba
)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,Accuracy,Precision,Recall,F1,ROC-AUC
0,LogisticRegression,linear_model,"{'C': 1.0, 'class_weight': None, 'dual': False...","Логистическая регрессия, стандартные параметры",0.9970,0.9970,0.9970,0.9970,None
1,LinearSVC,linear_model,"{'C': 1.0, 'class_weight': None, 'dual': 'auto...",линейная SVC,0.9992,0.9992,0.9992,0.9992,None


## SGDClassifier

## LinearDiscriminantAnalysis

## Итоги